In [15]:
import os
import sys

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

from openai import OpenAI
from secretstuff.secret import OPENAI_API_KEY, OPENAI_ORG_ID, OPENAI_PROJ_ID
from services.mongodb import catalogue
from services.metadata import get_catalogue_metadata
from pydantic import BaseModel
import random
import json

Connected to MongoDB server. Server info:
{'version': '8.0.4', 'gitVersion': 'bc35ab4305d9920d9d0491c1c9ef9b72383d31f9', 'modules': ['enterprise'], 'allocator': 'tcmalloc-google', 'javascriptEngine': 'mozjs', 'sysInfo': 'deprecated', 'versionArray': [8, 0, 4, 0], 'bits': 64, 'debug': False, 'maxBsonObjectSize': 16777216, 'storageEngines': ['devnull', 'inMemory', 'queryable_wt', 'wiredTiger'], 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1739499303, 26), 'signature': {'hash': b"\x15\x8a`U\x1f\xb8\xe7\xdf\x03\xd5'\x01\xd5\xd0\xb1I\xe8\xc5D\x98", 'keyId': 7432374265015435492}}, 'operationTime': Timestamp(1739499303, 26)}


In [100]:
create_outfit_tool = {
      "type": "function",
      "function": {
        "name": "create_outfit",
        "description": "Assembles an outfit consisting of top, bottom, and shoes with specified attributes.",
        "parameters": {
          "type": "object",
          "required": [
            "top",
            "shoes",
            "bottom"
          ],
          "properties": {
            "top": {
              "type": "object",
              "required": [
                "color",
                "material",
                "other_tags",
                "clothing_type"
              ],
              "properties": {
                "color": {
                  "type": "string",
                  "description": "Color of the top garment"
                },
                "material": {
                  "type": "string",
                  "description": "Material from which the top is made"
                },
                "other_tags": {
                  "type": "array",
                  "description": "List of tags describing the style of the top",
                  "items": {
                    "type": "string"
                  }
                },
                "clothing_type": {
                  "type": "string",
                  "description": "Type of clothing for the top"
                }
              },
              "additionalProperties": False
            },
            "shoes": {
              "type": "object",
              "required": [
                "color",
                "material",
                "other_tags",
                "clothing_type"
              ],
              "properties": {
                "color": {
                  "type": "string",
                  "description": "Color of the shoes"
                },
                "material": {
                  "type": "string",
                  "description": "Material from which the shoes are made"
                },
                "other_tags": {
                  "type": "array",
                  "description": "List of tags describing the style of the shoes",
                  "items": {
                    "type": "string"
                  }
                },
                "clothing_type": {
                  "type": "string",
                  "description": "Type of shoes"
                }
              },
              "additionalProperties": False
            },
            "bottom": {
              "type": "object",
              "required": [
                "color",
                "material",
                "other_tags",
                "clothing_type"
              ],
              "properties": {
                "color": {
                  "type": "string",
                  "description": "Color of the bottom garment"
                },
                "material": {
                  "type": "string",
                  "description": "Material from which the bottom is made"
                },
                "other_tags": {
                  "type": "array",
                  "description": "List of tags describing the style of the bottom",
                  "items": {
                    "type": "string"
                  }
                },
                "clothing_type": {
                  "type": "string",
                  "description": "Type of clothing for the bottom"
                }
              },
              "additionalProperties": False
            }
          },
          "additionalProperties": False
        },
        "strict": True
      }
    }

In [101]:
class ClothingTag(BaseModel):  # For catalogue
    clothing_type: str
    color: str
    material: str
    other_tags: list[str]

In [102]:
class UserPersona(BaseModel):
    age: int
    gender: str
    height: int
    skin_tone: str
    style: list[str]

    #recommendations: {'item':ClothingTag, recommended: [{'top':clothingTag, 'bottom':clothingTag, 'shoes':clothingTag}]}
    recommendations: dict[str, list[dict[str, ClothingTag]] | ClothingTag]

    #preferences: {'tops': ["casual tee shirts","textual graphics tee shirts"],'bottoms': ["jeans","trousers"],'shoes': ["sneakers","boots"]}}
    #idea is to populate this when the user gives fedback on the recommendations, keep maybe 3 preferences for each category -> to be updated with each feedback
    preferences: dict[str, list[str]]

In [103]:
openai_client = OpenAI(
    organization=OPENAI_ORG_ID,
    project=OPENAI_PROJ_ID,
    api_key=OPENAI_API_KEY
)

In [122]:
def generate_outfit_recommendations(item: ClothingTag, additional_prompt: str,user) -> ClothingTag:
    age = user.age
    gender = user.gender
    skin_tone = user.skin_tone
    style = user.style
    preferences = user.preferences

    #making a nice sentence about user preferences to inclcude in prompt
    parts = []
    if preferences == {}:
        user_preferences = ""
    else:
        if preferences['tops']:
            parts.append(f"{', '.join(preferences['tops'])}")
        if preferences['bottoms']:
            parts.append(f"{', '.join(preferences['bottoms'])}")
        if preferences['shoes']:
            parts.append(f"{', '.join(preferences['shoes'])}")
        user_preferences = f"The user prefers wearing: {', '.join(parts)}."

    #if there is no additional prompt, we don't want to include it in the prompt
    if not additional_prompt.strip():
        item_description = f"{item.color} {item.material} {item.clothing_type} ({item.other_tags}). Additional prompt: {additional_prompt}"
    else:
        item_description = f"{item.color} {item.material} {item.clothing_type} ({item.other_tags})"

    system_message = f"You are a fashion stylist creating an outfit for a {age} year old {skin_tone} skin {gender} who likes wearing {style} outfits. \
        {user_preferences} \
        Given a description of a clothing item (example: white polo tee shirt with black printed text) recommend complementary clothes to complete the outfit. \
        Follow these steps to recommend an outfit: 1) consider the user persona mentioned to analyse the style of clothes to be recommend.\
        2) if the given item is a top (tee shirt, polo, shirt, dress, tank top etc), give recommendations for bottoms (pants, shorts, trousers, jeans skirts, leggings etc) and for shoes. \
        If the given item is a shoe, give recommendations for tops and bottoms. If the given item is a bottom, give recommendations for the top and shoes. \
        You may also be given additional prompts in text to constrain the style of the matched item. \
        Compile the output into a JSON which contains the description of all the items in the completed outfit. Generate 3 such outfits. "
    
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": item_description},
            ],
        tools = [create_outfit_tool],
        tool_choice = {"type": "function", "function": {"name": "create_outfit"}})
    
    parsed_response = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
    top = parsed_response['top']
    bottom = parsed_response['bottom']
    shoes = parsed_response['shoes']

    top = ClothingTag(**{**top, 'other_tags': list(top['other_tags'])})
    bottom = ClothingTag(**{**bottom, 'other_tags': list(bottom['other_tags'])})
    shoes = ClothingTag(**{**shoes, 'other_tags': list(shoes['other_tags'])})


    # recommended = {'item':item, recommended: [{'top':top, 'bottom':bottom, 'shoes':shoes}]}
    if user.recommendations == {}:
        recommended = []
    else:
        recommended = user.recommendations['recommended']

    # #we only want to keep information about the last 3 recommendations
    if len(recommended) < 3:
        recommended.append({'top': top, 'bottom': bottom, 'shoes': shoes})
    else:
        recommended.pop(0)
        recommended.append({'top': top, 'bottom': bottom, 'shoes': shoes})

    user.recommendations = {'item': item, 'recommended': recommended}
    return top, bottom, shoes

In [123]:
item = ClothingTag(clothing_type="tee shirt", color="white", material="cotton", other_tags=["printed text"])
user = UserPersona(age=23, gender="male", height=180, skin_tone="light-brown", style=["casual", "streetwear"], recommendations={}, preferences={})

user.preferences

{}

In [124]:
outfit = generate_outfit_recommendations(item,"",user)

In [125]:
outfit

(ClothingTag(clothing_type='t-shirt', color='white', material='cotton', other_tags=['printed text']),
 ClothingTag(clothing_type='jeans', color='light-blue', material='denim', other_tags=['casual', 'streetwear', 'distressed']),
 ClothingTag(clothing_type='sneakers', color='black', material='canvas', other_tags=['casual', 'streetwear']))